In [2]:
import tensorflow as tf
import pickle
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

In [4]:
## Load the trained model , scaler pickle ,one hot 

model = load_model('model.h5')

## Load the encoder and scaler 

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    one_hot_encoder_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:

    scaler = pickle.load(file)




In [8]:
print(one_hot_encoder_geo.categories_)


[array(['France', 'Germany', 'Spain'], dtype=object)]


In [9]:
print(label_encoder_gender.classes_)


['Female' 'Male']


In [16]:
import pickle
import pandas as pd

# Load the one-hot encoder for Geography
with open("one_hot_encoder_geo.pkl", "rb") as file:
    one_hot_encoder_geo = pickle.load(file)

# Load the label encoder for Gender
with open("label_encoder_gender.pkl", "rb") as file:
    label_encoder_gender = pickle.load(file)

# Example input data
input_data = {
    'CreditScore': 619,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 42,
    'Tenure': 2,
    'Balance': 0,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 101348.88
}

# Convert to DataFrame
input_df = pd.DataFrame([input_data])

# Encode 'Gender' (Label Encoding)
input_df['Gender'] = label_encoder_gender.transform(input_df[['Gender']])

# Encode 'Geography' (One-Hot Encoding)
geo_transformed = one_hot_encoder_geo.transform(input_df[['Geography']])
geo_columns = one_hot_encoder_geo.categories_[0]

# Create DataFrame for one-hot encoded values
geo_df = pd.DataFrame(geo_transformed, columns=geo_columns, index=input_df.index)

# Concatenate with the main DataFrame and drop original categorical columns
input_df = pd.concat([input_df, geo_df], axis=1).drop(columns=['Geography'])

print("Final Encoded DataFrame:")
print(input_df)


Final Encoded DataFrame:
   CreditScore  Gender  Age  Tenure  Balance  NumOfProducts  HasCrCard  \
0          619       0   42       2        0              1          1   

   IsActiveMember  EstimatedSalary  France  Germany  Spain  
0               1        101348.88     1.0      0.0    0.0  


e:\csv file\venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [18]:
print("Columns in input_df:", input_df.columns.tolist())


Columns in input_df: ['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'France', 'Germany', 'Spain']


In [20]:
import pickle
import pandas as pd

# Load the scaler
with open("scaler.pkl", "rb") as file:
    scaler = pickle.load(file)

# Ensure input_df has the same column order as during training
expected_columns = scaler.feature_names_in_

# Reorder and add missing columns (fill with 0 if missing)
input_df = input_df.reindex(columns=expected_columns, fill_value=0)

# Apply scaling
scaled_input = scaler.transform(input_df)

# Convert scaled data back to a DataFrame (optional)
scaled_df = pd.DataFrame(scaled_input, columns=input_df.columns)

print("Scaled DataFrame:")
print(scaled_df)


Scaled DataFrame:
     Gender       Age    Tenure   Balance  NumOfProducts  HasCrCard  \
0 -0.774597  0.344214 -0.811503 -1.263113      -1.459601   0.377964   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0        1.290994         0.339778         -1.290994          -0.377964   

   Geography_Spain  
0         -0.57735  


In [21]:
## predicting whether the customer is likely to exit or not

prediction = model.predict(scaled_input)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step


array([[0.26572576]], dtype=float32)

In [22]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.26572576)

In [23]:
if prediction_proba>0.5:
    print("Customer is likely to exit ")
else:
    print("Customer is not likely to exit")

Customer is not likely to exit
